<a href="https://colab.research.google.com/github/MattIzon/16010269_DataAnalytics/blob/main/3_Crime_Aquisition_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set-up
from google.colab import auth, files
auth.authenticate_user()
print('Authenticated')

from google.cloud import bigquery
import pandas as pd
pd.set_option('display.max_columns', None)

Authenticated


In [2]:
# Use big query to get the crime data
client = bigquery.Client(project='assignment-305921')

crime = []

for year in range(2015, 2020):
  crime.append(client.query('''
  SELECT date, district, ward
  FROM `bigquery-public-data.chicago_crime.crime`
  WHERE year = {}
  ORDER BY date
  '''.format(year)).to_dataframe())

In [3]:
# Check for missing data
index = 0
for year in crime:
  print('Missing Data ', 2015 + index, ':')
  for column in year:
    print('  ', column, ': ', year[column].isna().any())
  index += 1
  print()

Missing Data  2015 :
   date :  False
   district :  False
   ward :  True

Missing Data  2016 :
   date :  False
   district :  False
   ward :  False

Missing Data  2017 :
   date :  False
   district :  True
   ward :  True

Missing Data  2018 :
   date :  False
   district :  False
   ward :  True

Missing Data  2019 :
   date :  False
   district :  False
   ward :  True



In [6]:
# Fill missing district in 2017
ward = crime[2].loc[crime[2]['district'].isnull(), 'ward'].values[0]
district = crime[2].loc[crime[2]['ward'] == ward]['district'].mode().values[0]
crime[2].loc[crime[2]['district'].isnull(), 'district'] = district

In [7]:
# Convert the datetime to date column
for year in crime:
  year['date'] = pd.to_datetime(year['date']).dt.date

In [8]:
# Add day_of_week column containing the day of the week: Monday 1 ... Sunday 7
jan_01 = [4, 5, 7, 1, 2]

def get_date_day(date_str):
  return date_str[8:]

for year in range(len(jan_01)):
  curr_date_day = '01'
  curr_d_o_w = jan_01[year]
  d_o_w = []

  for date_str in crime[year]['date']:
    date_day = get_date_day(date_str.strftime('%Y-%m-%d'))
    if date_day != curr_date_day:
      curr_date_day = date_day
      curr_d_o_w += 1
      if curr_d_o_w == 8:
        curr_d_o_w = 1
    d_o_w.append(curr_d_o_w)
  crime[year]['day_of_week'] = d_o_w

In [9]:
for year in crime:
  year['crime_count'] = year.date.map(year.date.value_counts())

In [11]:
# Drop ward
no_use = ['ward']
for year in crime:
  year.drop(no_use, axis=1, inplace=True)

In [13]:
# Count crimes per day in each district
districts = []
for year in crime:
  districts.append(year.groupby(['date', 'day_of_week','district'])['date'].count().reset_index(name='crime_count'))

In [16]:
# Drop district
no_use = ['district']
for year in crime:
  year.drop(no_use, axis=1, inplace=True)

In [ ]:
# Save district data to CSV files.

names = ['c_district_2015', 'c_district_2016', 'c_district_2017', 'c_district_2018', 'c_district_2019']

for index in range(len(names)):
  districts[index].to_csv('{}.csv'.format(names[index])) 
  files.download('{}.csv'.format(names[index]))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save day data to CSV files.

names = ['c_day_2015', 'c_day_2016', 'c_day_2017', 'c_day_2018', 'c_day_2019']

for index in range(len(names)):
  crime[index].to_csv('{}.csv'.format(names[index])) 
  files.download('{}.csv'.format(names[index]))